In [1]:
from common_imports import *
from functions import (
    create_subset,
    filter_datasets_for_use_case,
    split_datasets,
    unique_elements,
    PerplexityLoggingCallback,
    print_trainable_parameters,
    CustomDataset,
    get_sequences,
    evaluate,
    CustomTrainer
)

/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-02 21:07:51.574984: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115_nocublaslt.so...


/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('-I/root')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('2')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('-L/root/boost_1_75_0/stage/lib')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/var/lib/snap')}
  warn(msg)
/mnt/distvol/bitsandbytes/lib/python3.9/site-packages/bitsandbytes/cuda_setup

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("resources/train.json", "r") as f:
    args = json.load(f)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=args["load_in_4bit"],
    bnb_4bit_use_double_quant=args["bnb_4bit_use_double_quant"],
    bnb_4bit_quant_type=args["bnb_4bit_quant_type"],
    bnb_4bit_compute_dtype=eval(args["bnb_4bit_compute_dtype"])
)

lora_config = LoraConfig(
    r=args["lora_r"],
    lora_alpha=args["lora_alpha"],
    lora_dropout=args["lora_dropout"],
    bias=args["lora_bias"],
    task_type=args["lora_task_type"]
)

device_map = {"": 0}


In [12]:
infer_peft_config = PeftConfig.from_pretrained('bitsft')

infer_model = AutoModelForCausalLM.from_pretrained(
        infer_peft_config.base_model_name_or_path,
        quantization_config=bnb_config, device_map=device_map
)

#enabling raises memory requirements, but much faster inference
infer_model.gradient_checkpointing_enable()
infer_model = prepare_model_for_kbit_training(infer_model)
infer_model = get_peft_model(infer_model, lora_config)
print_trainable_parameters(infer_model)
#only necessary when use with the above
infer_model.model.config.use_cache = True


trainable params: 294912 || all params: 83026176 || trainable%: 0.3552036408373186


In [13]:
tokenizer = AutoTokenizer.from_pretrained(args['model_id'])
tokenizer.pad_token = tokenizer.eos_token


In [15]:

query_text = (
    "Context:\n"
    "Life can be difficult, yet rewarding.\n"
    "Prompt:\n"
    "What meaning can be found in life?\n"
    "Response:\n"
)
#attention_mask = torch.ones_like(input_ids)
generator = pipeline('text-generation', model=infer_model, tokenizer = tokenizer,
    min_length=50,
    max_length=128,
    temperature=1,
    #attention_mask=attention_mask,
    do_sample=True,
    top_k=50,
    top_p=1,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    num_beams=1,
    early_stopping=True)

results = generator(query_text)
print(results[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

Context:
Life can be difficult, yet rewarding.
Prompt:
What meaning can be found in life?
Response:
+
| The only way this was just fine. \.
 
.Jacket: ..........................

Thank you, .Please note. In case, they used this. After. Thank you......!!! As I, I am on the web. Now I will see some other iphone app. Or, on its time. But, before, in all the following, it is very little. And, the like. All at once
